In [1]:
from ipywidgets import Text, Button, Output, Dropdown, IntSlider, FloatSlider
from IPython.display import display
from IPython.core.display import HTML
import torch
from diffusers import StableDiffusionPipeline
import numpy as np

# Format the widget options
display(HTML("<style>.widget-label {text-align: left;}</style>"))
display(HTML("<style>.widget-text {text-align: left;}</style>"))

# Create a dropdown menu for model choice
model_choices = {
    "Stable Diffusion v1.4": "CompVis/stable-diffusion-v1-4",
    "Stable Diffusion v1.5": "CompVis/stable-diffusion-v1-5",
    "Stable Diffusion v2.0": "stabilityai/stable-diffusion-2-1-base"
}
model_dropdown = Dropdown(options=list(model_choices.keys()), value=list(model_choices.keys())[0], description='Model:', layout={'width': '600px'}, style={'description_width': 'initial'})

# Create an integer slider for the number of iterations
num_iterations_slider = IntSlider(min=1, max=100, value=50, description='Number of iterations:', layout={'width': '600px'}, style={'description_width': 'initial'})

# Create a float slider for the guidance scale
guidance_scale_slider = IntSlider(min=1, max=15.0, value=7.5, step=1, description='Guidance scale:', layout={'width': '600px'}, style={'description_width': 'initial'})

# Create an integer slider for the seed
seed_slider = IntSlider(min=0, max=100000, value=42, description='Seed:', layout={'width': '600px'}, style={'description_width': 'initial'})

# Create a float slider for the prompt strength
prompt_strength_slider = FloatSlider(min=0.0, max=1.0, value=0.8, step=0.01, description='Prompt strength:', layout={'width': '600px'}, style={'description_width': 'initial'})

# Create a dropdown menu for the sampling method
sampling_method_choices = {
    "Euler": "euler",
    "Euler-Ancestral": "euler_a",
    "K_LMS": "k_lms",
    "DDIM": "ddim",
    "DPM2": "dpm2",
    "PLMS": "plms"
}
sampling_method_dropdown = Dropdown(options=list(sampling_method_choices.keys()), value=list(sampling_method_choices.keys())[0], description='Sampling method:', layout={'width': '600px'}, style={'description_width': 'initial'})

# Create an integer slider for the latent space dimensions
latent_space_dimensions_slider = IntSlider(min=128, max=512, value=256, description='Latent space dimensions:', layout={'width': '600px'}, style={'description_width': 'initial'})

# Initialize the Stable Diffusion pipeline with the default model
model_id = model_choices[model_dropdown.value]
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# Check for Intel XPU device
device = torch.device("xpu" if torch.xpu.is_available() else "cpu")
print(f"Using device: {device}")
# Move the pipeline to the XPU if available, otherwise use CPU
pipe.to(device)

# Create a text input widget for the user prompt
prompt = Text(value='', placeholder='Enter your prompt', description='Prompt:', disabled=False, layout={'width': '500px'}, style={'description_width': 'initial'})

# Create a button to generate the image
button = Button(description='Generate Image', style={'button_color': 'blue', 'text_color': 'white'})

# Create an output widget to display the generated image
output = Output()

# Define a function to update the pipeline when the model is changed
def update_model(change):
    global pipe
    model_id = model_choices[change.new]
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe.to(device)

# Link the model dropdown to the update_model function
model_dropdown.observe(update_model, names='value')

# Define a function to generate the image when the button is clicked
def generate_image(b):
    with output:
        output.clear_output(wait=True)
        # Get the user prompt
        user_prompt = prompt.value
        # Get the number of iterations
        num_iterations = num_iterations_slider.value
        # Get the guidance scale
        guidance_scale = guidance_scale_slider.value
        # Get the seed
        seed = seed_slider.value
        # Get the prompt strength
        prompt_strength = prompt_strength_slider.value
        # Get the sampling method
        sampling_method = sampling_method_choices[sampling_method_dropdown.value]
        # Get the latent space dimensions
        latent_space_dimensions = latent_space_dimensions_slider.value
        # Generate the image using the Stable Diffusion pipeline
        torch.manual_seed(seed)
        torch.xpu.empty_cache()
        image = pipe(user_prompt, num_inference_steps=num_iterations, guidance_scale=guidance_scale, height=640, width=640, prompt_strength=prompt_strength).images[0]
        # Display the generated image
        display(image)

# Link the button to the generate_image function
button.on_click(generate_image)

# Display the widgets
display(prompt)
display(model_dropdown)
display(num_iterations_slider)
display(guidance_scale_slider)
display(seed_slider)
display(prompt_strength_slider)
display(sampling_method_dropdown)
display(latent_space_dimensions_slider)
display(button)
display(output)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Using device: xpu


Text(value='', description='Prompt:', layout=Layout(width='500px'), placeholder='Enter your prompt', style=Tex…

Dropdown(description='Model:', layout=Layout(width='600px'), options=('Stable Diffusion v1.4', 'Stable Diffusi…

IntSlider(value=50, description='Number of iterations:', layout=Layout(width='600px'), min=1, style=SliderStyl…

...

FloatSlider(value=0.8, description='Prompt strength:', layout=Layout(width='600px'), max=1.0, step=0.01, style…

Dropdown(description='Sampling method:', layout=Layout(width='600px'), options=('Euler', 'Euler-Ancestral', 'K…

IntSlider(value=256, description='Latent space dimensions:', layout=Layout(width='600px'), max=512, min=128, s…

Button(description='Generate Image', style=ButtonStyle(button_color='blue', text_color='white'))

Output()

#### Note
* If you run out of memory restart your kernel.

#### Iteration 
* This refers to the repetitive cycle starting from generating random noise based on text input. Each step eliminates some noise, and with an increase in the number of iteration steps, higher-quality images are produced

#### Guidance Scale
* A value of 0 gives random images and does not consider your prompt at all.
* Lower values give more loosely interpreted images.
* Higher values give images which represent your prompt more precisely. 
* Values from 5 to 15 seem to be the range consensus.

#### Seed

* This variable controls the random seed used to initialize the model's weights. Changing the seed can result in different output images, even with the same input prompt

#### Prompt Strength

* This variable controls the strength of the input prompt. A higher prompt strength can lead to more realistic images, but can also introduce artifacts.

#### Sampling Method

* This variable controls the sampling method used to generate images. Common sampling methods include Euler, Euler-Ancestral, and DDIM.

#### Latent Space Dimensions

* This variable controls the dimensionality of the latent space used to represent images. Increasing the latent space dimensions can lead to more detailed images, but can also increase computation time.